In [20]:
import os
import math
import random
import numpy as np
import pandas as pd
from operator import itemgetter
from collections import defaultdict
from abc import ABCMeta, abstractmethod
import time
U_global = 0.83

class Hash:
    __metaclass__ = ABCMeta

    def __init__(self):
        pass

    @abstractmethod
    def hash(self, vec):
        print "Hash.hash()"
        pass

    @staticmethod
    def distance(u, v):
        print "Hash.distance()"
        pass

    @staticmethod
    def combine(hashes):
        return str(hashes)

    
class L2Lsh(Hash):
    def __init__(self, r, d):
        self.r, self.d = r, d
        self.b = random.uniform(0, self.r)      # 0 < b < r
        self.Data = [random.gauss(0, 1) for i in xrange(self.d)]

    def hash(self, vec):	# hash family
        return int((np.dot(vec, self.Data) + self.b) / self.r)

    @staticmethod
    def distance(u, v):
        return sum((ux - vx)**2 for ux, vx in zip(u, v))**0.5

class Lshfunctions:
    'LSH Wrapper'

    def __init__(self, lsh_type, d, r = 2.5, k = 2, L = 2):
        self.type = lsh_type
        self.d, self.r, self.k, self.L, self.hash_tables = d, r, k, 0, []
        self.resize(L)

    def __get_hash_class__(self):
        return L2Lsh

    def create_hash_table(self):
        return L2Lsh(self.r, self.d)

    def resize(self, L):
        if L < self.L:
            self.hash_tables = self.hash_tables[:L]
        else:
            hash_funcs = [[self.create_hash_table() for h in xrange(self.k)] for l in xrange(self.L, L)]
            self.hash_tables.extend([(g, defaultdict(lambda:[])) for g in hash_funcs])
        self.L = L

    def hash(self, ht, data):
        return self.__get_hash_class__().combine([h.hash(data) for h in ht])

    def index(self, train_dp):
        self.train_dp = train_dp
        for ht, ct in self.hash_tables:
            for ix, p in enumerate(self.train_dp):
                ct[self.hash(ht, p)].append(ix)
        self.tot_touched = 0
        self.num_test_dp = 0

    def query(self, q, metric, max_results = 1):
        if 0 == max_results:
            max_results = 1
        candidates = set()
        for ht, ct in self.hash_tables:
            matches = ct.get(self.hash(ht, q), [])
            candidates.update(matches)
        self.tot_touched += len(candidates)
        self.num_test_dp += 1
        candidates = [(ix, metric(q, self.train_dp[ix])) for ix in candidates]
        candidates.sort(key=itemgetter(1))
        return candidates[:max_results]

    def get_avg_touched(self):
        return self.tot_touched / self.num_test_dp

    def distance(self, u, v):
        return __get_hash_class__.distance(u, v)

    
    
class L2LSH():
    def __init__(self, train_dp, test_dp, r_values = 2.5, num_neighbours = 1):
        kdata, qdata = len(train_dp[0]), len(test_dp[0])
        self.d = kdata
        self.train_dp = train_dp
        self.test_dp = test_dp
        self.r_values = r_values
        self.q_num = len(self.test_dp)
        self.num_neighbours = num_neighbours

    def linear(self, q, metric, max_results):
        candidates = [(ix, metric(q, p)) for ix, p in enumerate(self.train_dp)]
        temp = sorted(candidates, key=itemgetter(1))
        temp=temp[:max_results]
        return temp

    def run(self, type, k_vec = [2], l_vec = [2]):
        if 'l2' == type:
            metric = L2Lsh.distance
        exact_hits = [[ix for ix, dist in self.linear(q, metric, self.num_neighbours)] for q in self.test_dp]   #exact_hits是距离q最近的那些点（这是真的，精准的）
        for k in k_vec:
            lsh = Lshfunctions(type, self.d, self.r_values, k, 0)

            for L in l_vec:
                lsh.resize(L)
                lsh.index(self.train_dp)
                correct = 0
                num_query = 0
                recall = 0
                precision = 0
                noZeroCount = np.count_nonzero(self.test_dp)
                for q, hits in zip(self.test_dp, exact_hits):                   
                    lsh_hits = [ix for ix, dist in lsh.query(q, metric, self.num_neighbours)]
                    correct += int(lsh_hits == hits)
                    num_query+=1
                    recall += (float)(correct)/self.num_neighbours
                    precision += (float)(correct)/noZeroCount
                    correct = 0
                recall/=num_query
                precision/=num_query
                print "{0}\t{1}\t{2}\t{3}".format(L, k, float(precision), float(recall))

class ALSH_MIPS(L2LSH):
    'L2-ALSH for MIPS Test parameters'

    def __init__(self, train_dp, test_dp, r_values = 2.5, num_neighbours = 1, m = 3):
        kdata = len(train_dp[0])
        qdata = len(test_dp[0])
        self.m = m
        self.half_extend = [0.5 for i in xrange(self.m)]
        self.origin_train_dp = train_dp
        self.origin_test_dp = test_dp
        self.q_num = len(self.origin_test_dp)
        dmax_norm = max([math.sqrt(np.dot(dd, dd)) for dd in train_dp])
        dratio = float(U_global / dmax_norm)
        self.norm_train_dp = [[dratio * dx for dx in dd] for dd in self.origin_train_dp]
        norm_test_dp = []
        for qv in self.origin_test_dp:
            norm = math.sqrt(np.dot(qv, qv))
            ratio = float(U_global / norm)
            norm_test_dp.append([ratio * qx for qx in qv])
        
        self.norm_test_dp = norm_test_dp
        self.ext_train_dp = [(dv + [np.dot(dv, dv)**(i+1) for i in xrange(self.m)] + [0.5 for i in xrange(self.m)]) for dv in self.norm_train_dp]
        self.ext_test_dp = [(qv + [0.5 for i in xrange(self.m)] + [np.dot(qv, qv)**(i+1) for i in xrange(self.m)]) for qv in self.norm_test_dp]
        new_len = kdata + 2 * m
        L2LSH.__init__(self, self.ext_train_dp, self.ext_test_dp, r_values, num_neighbours)

    # MIPS
    def linear(self, q, metric, max_results):
        candidates = [(ix, np.dot(q, p)) for ix, p in enumerate(self.origin_train_dp)]
        temp = sorted(candidates, key=itemgetter(1), reverse=True)
        temp=temp[:max_results]
        return temp

    def run(self, type, k_vec = [2], l_vec = [2]):
        validate_metric, compute_metric = np.dot, L2Lsh.distance
        start = time.time()
        exact_hits = [[ix for ix, dist in self.linear(q, compute_metric, self.num_neighbours)] for q in self.origin_test_dp]
        exact_hits = [[ix for ix, dist in self.linear(q, validate_metric, self.num_neighbours)] for q in self.origin_test_dp]
        linear_time = time.time() - start
        print 'ALSH_MIPS ' + type + ' TEST:'
        print 'L\tk\tPrecision\trecall\tALSH time\tLinear time'

        for k in k_vec:
            lsh = Lshfunctions(type, self.d, self.r_values, k, 0)
            for L in l_vec:
                lsh.resize(L)
                lsh.index(self.ext_train_dp)
                correct = 0
                num_query = 0
                recall = 0
                precision = 0
                noZeroCount = np.count_nonzero(self.ext_test_dp)
                start = time.time()
                for q, hits in zip(self.ext_test_dp, exact_hits):
                    lsh_hits = [ix for ix, dist in lsh.query(q, metric, self.num_neighbours)]
                    correct = correct + int(lsh_hits == hits)
                    num_query+=1
                    recall += (float)(correct)/self.num_neighbours
                    precision += (float)(correct)/noZeroCount
                    correct = 0
                recall/=num_query
                precision/=num_query
                alsh_time = time.time() - start
                
                print "{0}\t{1}\t{2}\t{3}\t\t{4}\t\t{5}".format(L, k, float(precision), float(recall), alsh_time, linear_time)
    
    @staticmethod
    def RunOnParams(type, train_dp, test_dp, rand_num, num_neighbours):
        return ALSH_MIPS(train_dp, test_dp, rand_num, num_neighbours)

def lsh_test(train_dp, test_dp, rand_num, num_neighbours, mips = False):

    type = 'l2'
    tester = ALSH_MIPS.RunOnParams(type, train_dp, test_dp, rand_num, num_neighbours)
    args = {
                'type':      type,
                'k_vec':     [1, 2, 4, 8],
                'l_vec':     [2, 4, 8, 16, 32]
            }
    tester.run(**args)

    """

    type = 'cosine'
    tester = L2LSH.RunOnParams(type, mips, train_dp, test_dp, rand_num, num_neighbours)

    args = {
                'type':      type,
                'k_vec':    [1, 2, 4, 8],
                'l_vec':    [2, 4, 8, 16, 32]
            }
    tester.run(**args)

    """

if __name__ == "__main__":
    train_dpet = "../datasets" + os.path.sep + "ml-latest-small"
    name = "ratings.csv"
    ratings_df = pd.read_csv(train_dpet + os.path.sep + name, names= ["UserID", "MovieID", "Rating", "Timestamp"], header=0)
    ratings_df["UserID"] = pd.to_numeric(ratings_df["UserID"], errors='ignore')
    ratings_df["MovieID"] = pd.to_numeric(ratings_df["MovieID"], errors='ignore')
    ratings_df["Rating"] = pd.to_numeric(ratings_df["Rating"], errors='ignore')
    ratings_df["Timestamp"] = pd.to_numeric(ratings_df["Timestamp"], errors='ignore')
    R_df = ratings_df.pivot(index = 'UserID', columns ='MovieID', values = 'Rating').fillna(0)  
    R = R_df.as_matrix()
    user_ratings_mean = np.mean(R, axis = 1)
    R_demeaned = R - user_ratings_mean.reshape(-1, 1)
    TRAIN_SIZE = 0.80
    train_ratings = R_demeaned[:100]
    test_ratings = R_demeaned[100:120]
    test_r = np.array(test_ratings)
    train_r = np.array(train_ratings)
    num_neighbours = 1
    r_range = 10 * 0.3
    train_dp = train_r
    test_dp = test_r
    print("1")
    lsh_test(train_dp, test_dp, r_range, num_neighbours, True)

1
ALSH_MIPS l2 TEST:
L	k	Precision	recall	ALSH time	Linear time


NameError: global name 'metric' is not defined

We compute the precision and recall of the top-T items for T ∈ {1, 5, 10}, obtained from the sorted list
based on M atches. To compute this precision and recall, we start at the top of the ranked item list and walk
down in order. Suppose we are at the kth ranked item, we check if this item belongs to the gold standard top-T list. If it is one of the top-T gold standard item, then we increment the count of relevant seen by 1,
else we move to k + 1. By kth step, we have already seen k items, so the total items seen is k. The precision and recall at that point is then computed as: Precision =(relevant seen/k); Recall = (relevant seen/T)